# Article Page Views API Example
This example illustrates how to access page view data using the [Wikimedia REST API](https://www.mediawiki.org/wiki/Wikimedia_REST_API). This example shows how to request monthly counts of page views for one specific article. The API documentation, [pageviews/per-article](https://wikimedia.org/api/rest_v1/#/Pageviews%20data), covers additional details that may be helpful when trying to use or understand this example.

## License
This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.3 - August 16, 2024



In [19]:
# 
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests
import pandas as pd


The example relies on some constants that help make the code a bit more readable.

Change 
ARTICLE_TITLE lists and make it iterate through the list. 
Run desktop, mobile-app, mobile-web (API:Get number of page views for a page)
Date range from July 1 2015 - September 31 2024
Email to own

In [20]:
#########
#
#    CONSTANTS
#

# The REST API 'pageviews' URL - this is the common URL/endpoint for all 'pageviews' API requests
API_REQUEST_PAGEVIEWS_ENDPOINT = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/'

# This is a parameterized string that specifies what kind of pageviews request we are going to make
# In this case it will be a 'per-article' based request. The string is a format string so that we can
# replace each parameter with an appropriate value before making the request
API_REQUEST_PER_ARTICLE_PARAMS = 'per-article/{project}/{access}/{agent}/{article}/{granularity}/{start}/{end}'

# The Pageviews API asks that we not exceed 100 requests per second, we add a small delay to each request
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making a request to the Wikimedia API they ask that you include your email address which will allow them
# to contact you if something happens - such as - your code exceeding rate limits - or some other error 
REQUEST_HEADERS = {
    'User-Agent': '<uwnetid@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# Read the csv file to obtain the list of article titles under the "disease" column.
disease_csv = pd.read_csv("rare-disease_cleaned.AUG.2024.csv")
ARTICLE_TITLES = disease_csv["disease"]

# This template is used to map parameter values into the API_REQUST_PER_ARTICLE_PARAMS portion of an API request. The dictionary has a
# field/key for each of the required parameters. In the example, below, we only vary the article name, so the majority of the fields
# can stay constant for each request. Of course, these values *could* be changed if necessary.
ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "desktop",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015010100",   # start and end dates need to be set
    "end":         "2023040100"    # this is likely the wrong end date
}


The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore the parameter most likely to change is the article_title.

In [21]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageviews_per_article(article_title = None, 
                                  endpoint_url = API_REQUEST_PAGEVIEWS_ENDPOINT, 
                                  endpoint_params = API_REQUEST_PER_ARTICLE_PARAMS, 
                                  request_template = ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE,
                                  headers = REQUEST_HEADERS):

    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['article'] = article_title

    if not request_template['article']:
        raise Exception("Must supply an article title to make a pageviews request.")

    # Titles are supposed to have spaces replaced with "_" and be URL encoded
    # Nothing is safe to allow / to change to %2F
    article_title_encoded = urllib.parse.quote(request_template['article'].replace(' ','_'),safe='')
  
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [22]:
ARTICLE_TITLES = ARTICLE_TITLES[0:2]
desktop_views = []
for i in ARTICLE_TITLES:
    print("Getting pageview data for: ",i)
    views = request_pageviews_per_article(i)
    desktop_views.append(views)

print (desktop_views)


Getting pageview data for:  Klinefelter syndrome
Getting pageview data for:  Aarskog–Scott syndrome
[{'items': [{'project': 'en.wikipedia', 'article': 'Klinefelter_syndrome', 'granularity': 'monthly', 'timestamp': '2015070100', 'access': 'desktop', 'agent': 'user', 'views': 36798}, {'project': 'en.wikipedia', 'article': 'Klinefelter_syndrome', 'granularity': 'monthly', 'timestamp': '2015080100', 'access': 'desktop', 'agent': 'user', 'views': 33180}, {'project': 'en.wikipedia', 'article': 'Klinefelter_syndrome', 'granularity': 'monthly', 'timestamp': '2015090100', 'access': 'desktop', 'agent': 'user', 'views': 35882}, {'project': 'en.wikipedia', 'article': 'Klinefelter_syndrome', 'granularity': 'monthly', 'timestamp': '2015100100', 'access': 'desktop', 'agent': 'user', 'views': 39887}, {'project': 'en.wikipedia', 'article': 'Klinefelter_syndrome', 'granularity': 'monthly', 'timestamp': '2015110100', 'access': 'desktop', 'agent': 'user', 'views': 40749}, {'project': 'en.wikipedia', 'arti

In [5]:
#print(json.dumps(views,indent=4))
print(f"Collected {len(views['items'])} months of pageview data")
for month in views['items']:
    print(json.dumps(month,indent=4))

Collected 94 months of pageview data
{
    "project": "en.wikipedia",
    "article": "Northern_flicker",
    "granularity": "monthly",
    "timestamp": "2015070100",
    "access": "desktop",
    "agent": "user",
    "views": 4018
}
{
    "project": "en.wikipedia",
    "article": "Northern_flicker",
    "granularity": "monthly",
    "timestamp": "2015080100",
    "access": "desktop",
    "agent": "user",
    "views": 3116
}
{
    "project": "en.wikipedia",
    "article": "Northern_flicker",
    "granularity": "monthly",
    "timestamp": "2015090100",
    "access": "desktop",
    "agent": "user",
    "views": 4802
}
{
    "project": "en.wikipedia",
    "article": "Northern_flicker",
    "granularity": "monthly",
    "timestamp": "2015100100",
    "access": "desktop",
    "agent": "user",
    "views": 5373
}
{
    "project": "en.wikipedia",
    "article": "Northern_flicker",
    "granularity": "monthly",
    "timestamp": "2015110100",
    "access": "desktop",
    "agent": "user",
    "vie

Above output should show dictionaries with views per month